## Step 1: Generate OAuth URI

In [3]:
from google.cloud import secretmanager
import json

client = secretmanager.SecretManagerServiceClient()


def get_secret(secret_id, project_id):
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


########################################################
# UPDATE THIS
########################################################
PROJECT_ID = "hello-world-418507"
PROJECT_NUMBER = "671247654914"

CLIENT_JSON = get_secret("AGENTSPACE_WEB_SECRET_JSON", PROJECT_ID)
client_config = json.loads(CLIENT_JSON)

AUTH_ID = get_secret("AGENTSPACE_WEB_AUTH_ID", PROJECT_ID)
print(AUTH_ID)

demo-agentspace-auth-20250727


In [4]:
import google_auth_oauthlib.flow

# Scope: https://developers.google.com/identity/protocols/oauth2/scopes
flow = google_auth_oauthlib.flow.Flow.from_client_config(
    client_config,
    scopes=[
        "https://www.googleapis.com/auth/gmail.modify",
        "https://www.googleapis.com/auth/gmail.labels",
        "https://www.googleapis.com/auth/calendar.events",
        "https://www.googleapis.com/auth/bigquery",
    ],
)

flow.redirect_uri = "https://vertexaisearch.cloud.google.com/oauth-redirect"

authorization_url, state = flow.authorization_url(
    access_type="online",  # Recommended, enable offline access so that you can refresh an access token without re-prompting
    include_granted_scopes="true",
    prompt="consent",
)
print("OAUTH_AUTH_URI:")
print(authorization_url)

OAUTH_AUTH_URI:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=671247654914-onsqf0obdfnkdpr0uj59hruev41u54i1.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fvertexaisearch.cloud.google.com%2Foauth-redirect&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.labels+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.events+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=MPQ55snuLwmgqzOjkhPKVNF1ciOFX0&access_type=online&include_granted_scopes=true&prompt=consent


## Step 2: Create authorization

In [ ]:
import subprocess
import requests

# Get access token
access_token = subprocess.check_output(
    ["gcloud", "auth", "print-access-token"], text=True
).strip()

# Prepare the request data
data = {
    "name": f"projects/{PROJECT_NUMBER}/locations/global/authorizations/{AUTH_ID}",
    "serverSideOauth2": {
        "clientId": client_config["web"]["client_id"],
        "clientSecret": client_config["web"]["client_secret"],
        "authorizationUri": authorization_url,
        "tokenUri": client_config["web"]["token_uri"],
    },
}

# Make the API request
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_NUMBER}/locations/global/authorizations?authorizationId={AUTH_ID}"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_NUMBER,
}


response = requests.post(url, headers=headers, json=data)
print(f"Status Code: {response.status_code}")
print(response.text)


## Step 3: Delete authorization

In [ ]:
import subprocess
import requests

access_token = subprocess.check_output(
    ["gcloud", "auth", "print-access-token"], text=True
).strip()

# Make the DELETE request
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_NUMBER}/locations/global/authorizations/{AUTH_ID}"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_NUMBER,
}

response = requests.delete(url, headers=headers)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.text}")
